# Proyecto final Machine Learning

Jorge Ruizvisfocri

Emilio Martinez

# Descripción del proyecto

El siguiente proyecto busca clasificar imágenes de Carcinomas Ductales Invasivos (IDC en inglés) extraidas de muestras de pacientes con cancer de mama.

La base de datos fue tomada de https://www.kaggle.com/datasets/paultimothymooney/breast-histopathology-images

Se propone utilizar una red neuronal convolucional para resolver el problema de clasificación.

# Paqueterías de trabajo

In [28]:
## Paquetes de ciencias de datos
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
import math
import random

## Paquetes de lectura de imagenes
from pathlib import Path
import os
import glob

## Paquetes de imágenes
from PIL import Image
import cv2

##Otros
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Datos de trabajo

In [30]:
# Directorio de imágenes
data_dir = Path("C:/Users/permi/OneDrive - CIDE/CIDE/4to semestre/ML/imagenes/IDC_regular_ps50_idx5") ## Directorio de datos

Dado que todos los pacientes tienen muestras con tumores malignos y benignos, la aleatorización puede realizarse a nivel de imagen o a nivel de paciente.

Bajo el supuesto de que podría existir correlación de algún tipo entre las imágenes pertenecientes a un mismo paciente, creemos que sería interesante aleatorizar a nivel de paciente, para introducirle a la red la información de personas que jamás ha visto.

In [31]:
### Aleatorización a nivel de paciente
#### Obtenemos la lista de pacientes
px = [f for f in data_dir.iterdir() if f.is_dir()]

In [32]:
### Obtenemos el número equivalente al porcentaje deseado
k = math.ceil( len(px) * 80 // 100)

In [ ]:
### Hacemos la muestra de pacientes para entrenamiento y prueba

In [33]:
### fijamos la semilla para tener reproductibilidad
random.seed(4352)

In [34]:
train_id = random.sample(px,k)

In [35]:
test_id = set(px) - set(train_id)

In [36]:
### Limpieza
del data_dir
del px
del k

In [37]:
### cargamos files de entrenamiento
train_images_files = []
for p in tqdm(train_id):
    img_lst = list(p.rglob("*.png"))
    train_images_files.extend(img_lst)

100%|██████████| 223/223 [00:02<00:00, 83.15it/s] 


In [38]:
test_images_files = []
for p in tqdm(test_id):
    img_lst = list(p.rglob("*.png"))
    test_images_files.extend(img_lst)

100%|██████████| 56/56 [00:00<00:00, 78.60it/s]


In [39]:
### Revisamos que las direcciones de las imágenes estén bien cargadas
print(
    f"Número de imagenes\n"
    "-----------------\n"
    f"Total: {len(train_images_files) + len(test_images_files) }\n"  # 277,524 tiles
    "-----------------\n"
    f"Entrenamiento: {len(train_images_files) }\n"
    "-----------------\n"
    f"Prueba: {len(test_images_files) }\n"
)

Número de imagenes
-----------------
Total: 277524
-----------------
Entrenamiento: 220389
-----------------
Prueba: 57135



In [40]:
### Lipieza
del train_id
del test_id

## Preparamos las etiquetas

In [41]:
## y_entrenamiento
### Preparamos las etiquetas
y_train = []
for name in tqdm(train_images_files):
    etiqueta = str(name)[-5]
    etiqueta_num = int(etiqueta)
    y_train.append(etiqueta_num)

100%|██████████| 220389/220389 [00:00<00:00, 722156.01it/s]


In [74]:
y_train_array = np.asarray(y_train,dtype=object)

In [42]:
y_test = []
for name in tqdm(test_images_files):
    etiqueta = str(name)[-5]
    etiqueta_num = int(etiqueta)
    y_test.append(etiqueta_num)

100%|██████████| 57135/57135 [00:00<00:00, 712254.41it/s]


In [75]:
y_test_array = np.asarray(y_test,dtype=object)

In [43]:
### Revisamos números de positivos en conjuntos
print(
    f"Positivos totales\n"
    "-----------------\n"
    f"Total: {sum(y_train) + sum(y_test)}\n"  # 277,524 tiles
    "-----------------\n"
    f"Entrenamiento: {sum(y_train) }\n"
    "-----------------\n"
    f"Prueba: {sum(y_test) }\n"
     "------------------------\n"
    f"Positivos porcentajes\n"
    "-----------------\n"
    f"Total: { round((sum(y_train) + sum(y_test))/ (len(y_train) + len(y_test)),2)  }\n"  # 277,524 tiles
    "-----------------\n"
    f"Entrenamiento: {round(sum(y_train)/ len(y_train),2) }\n"
    "-----------------\n"
    f"Prueba: {round(sum(y_test)/ len(y_test),2) }\n"
)

Positivos totales
-----------------
Total: 78786
-----------------
Entrenamiento: 64466
-----------------
Prueba: 14320
------------------------
Positivos porcentajes
-----------------
Total: 0.28
-----------------
Entrenamiento: 0.29
-----------------
Prueba: 0.25



## Preparamos las imágenes

In [44]:
### Cargamos de imágenes de entrenamiento en una lista
dataset_img_train = list()
for img in tqdm(train_images_files):
    image = Image.open(img)
    image=image.resize((50,50))
    numpydata = np.asarray(image)
    dataset_img_train.append(numpydata)

100%|██████████| 220389/220389 [02:16<00:00, 1610.94it/s]


In [45]:
### Convertimos la lista en tensor
dataset_img_train_array = np.asarray(dataset_img_train,dtype=object)

In [46]:
### Limpieza
del train_images_files
del etiqueta
del etiqueta_num
del dataset_img_train
del numpydata

In [47]:
dataset_img_train_array.shape ## Debe darnos un vector de tamaño (n,a,b,c)

(220389, 50, 50, 3)

In [48]:
### Aplanamos los datos
dataset_img_train_array_plana = dataset_img_train_array.astype('float32') / 255

In [49]:
dataset_img_train_array_plana[0:3] ## Vemos las primeras entradas para verificar que estén aplanados los datos

array([[[[0.89411765, 0.63529414, 0.7411765 ],
         [0.8627451 , 0.5921569 , 0.7058824 ],
         [0.8901961 , 0.5764706 , 0.7058824 ],
         ...,
         [0.9254902 , 0.8       , 0.85882354],
         [0.92156863, 0.7647059 , 0.83137256],
         [0.9019608 , 0.7372549 , 0.80784315]],

        [[0.8509804 , 0.56078434, 0.68235296],
         [0.8392157 , 0.5137255 , 0.6627451 ],
         [0.8156863 , 0.48235294, 0.6117647 ],
         ...,
         [0.9490196 , 0.9019608 , 0.91764706],
         [0.9411765 , 0.8862745 , 0.9137255 ],
         [0.9019608 , 0.7137255 , 0.8039216 ]],

        [[0.89411765, 0.6431373 , 0.7529412 ],
         [0.88235295, 0.62352943, 0.7137255 ],
         [0.88235295, 0.5764706 , 0.7176471 ],
         ...,
         [0.8862745 , 0.654902  , 0.7529412 ],
         [0.9254902 , 0.7647059 , 0.84705883],
         [0.9254902 , 0.79607844, 0.8392157 ]],

        ...,

        [[0.92156863, 0.7921569 , 0.85490197],
         [0.9254902 , 0.8039216 , 0.84705883]

In [50]:
del dataset_img_train_array

In [51]:
## Revisamos que tengan la misma longitud para el conjunto de entrenamiento
leny_train = len(y_train)
lenset_train = len(dataset_img_train_array_plana)

if leny_train == lenset_train:
    print("El tamaño del vector de resultados y de las imágenes es el mismo en el conjunto de entrenamiento")
else:
    print("El tamaño del vector de resultados y las imágenes no coincide en el conjunto de entrenamiento")

El tamaño del vector de resultados y de las imágenes es el mismo en el conjunto de entrenamiento


In [52]:
### Cargamos imágenes de prueba en una lista
dataset_img_test = list()
for img in tqdm(test_images_files):
    image = Image.open(img)
    image=image.resize((50,50))
    numpydata = np.asarray(image)
    numpydata_plana = numpydata.astype('float32') / 255
    dataset_img_test.append(numpydata_plana)

100%|██████████| 57135/57135 [00:37<00:00, 1538.23it/s]


In [53]:
### Convertimos la lista en tensor
dataset_img_test_array = np.asarray(dataset_img_test,dtype=object)

In [54]:
### Limpieza
del test_images_files
del dataset_img_test
del image
del numpydata
del numpydata_plana

In [55]:
dataset_img_test_array.shape ## Debe darnos un vector de tamaño (n,a,b,c)

(57135, 50, 50, 3)

In [56]:
### Aplanamos los datos
dataset_img_test_array_plana = dataset_img_test_array.astype('float32') / 255

In [57]:
dataset_img_test_array_plana[0:3] ## Vemos las primeras entradas para verificar que estén aplanados los datos

array([[[[0.00350634, 0.00330642, 0.00344483],
         [0.00366013, 0.0035371 , 0.00358324],
         [0.00372165, 0.00370627, 0.00370627],
         ...,
         [0.00367551, 0.00358324, 0.00367551],
         [0.00376778, 0.00370627, 0.00370627],
         [0.00370627, 0.00372165, 0.00381392]],

        [[0.0035371 , 0.00322953, 0.00339869],
         [0.00372165, 0.00362937, 0.00367551],
         [0.00372165, 0.00372165, 0.00372165],
         ...,
         [0.0037524 , 0.00369089, 0.00370627],
         [0.00370627, 0.00364475, 0.00372165],
         [0.00372165, 0.00361399, 0.00361399]],

        [[0.00362937, 0.00342945, 0.00350634],
         [0.00369089, 0.00358324, 0.00369089],
         [0.00372165, 0.00372165, 0.00369089],
         ...,
         [0.00362937, 0.00335256, 0.00355248],
         [0.00352172, 0.00324491, 0.00339869],
         [0.00372165, 0.00355248, 0.00364475]],

        ...,

        [[0.00372165, 0.00366013, 0.00370627],
         [0.00370627, 0.00367551, 0.00370627]

In [27]:
## Revisamos que tengan la misma longitud para el conjunto de prueba
leny_test = len(y_test)
lenset_test = len(dataset_img_test_array_plana)

if leny_test == lenset_test:
    print("El tamaño del vector de resultados y de las imágenes es el mismo en el conjunto de prueba")
else:
    print("El tamaño del vector de resultados y las imágenes no coincide en el conjunto de prueba")

El tamaño del vector de resultados y de las imágenes es el mismo en el conjunto de prueba


# Modelo

## Conjunto de entrenamiento y validación

In [76]:
### Creamos conjuntos de validación y entrenamiento
X_train, X_val, y_train2, y_val = train_test_split(dataset_img_train_array_plana, y_train_array, test_size=0.33, random_state=42)

In [ ]:
### Limpieza
del dataset_img_train_array_plana

## Arquitectura del modelo

In [77]:
### Usamos modelo de la tarea 11
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same', input_shape=(50, 50, 3)))
model.add(Conv2D(32, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
#model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
#model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [78]:
model.compile(Adam(learning_rate=.01),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [79]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 50, 50, 32)        896       
                                                                 
 conv2d_11 (Conv2D)          (None, 50, 50, 32)        9248      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 25, 25, 32)       0         
 2D)                                                             
                                                                 
 conv2d_12 (Conv2D)          (None, 25, 25, 64)        18496     
                                                                 
 conv2d_13 (Conv2D)          (None, 25, 25, 64)        36928     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 12, 12, 64)       0         
 2D)                                                  

## Entrenamiento del modelo

In [73]:
len(y_train2)

147660

In [1]:
## Entrenamos el modelo
history = model.fit(X_train, y_train2, batch_size=256, epochs=2, validation_data=(X_val, y_val), shuffle=True)

NameError: name 'model' is not defined

## Gráficas de pérdida y precisión en el entrenamiento

### Pérdida

In [ ]:
for hist, lr in zip(history, learning_rates):
    loss = hist.history['loss']
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, 'o-', label=lr)
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

### Precisión

In [ ]:
for hist, lr in zip(history, learning_rates):
    loss = hist.history['accuracy']
    epochs = range(1, len(loss) + 1)
    plt.semilogy(epochs, loss, 'o-', label=lr)
plt.title('Training accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Predicciones

In [ ]:
pred = model.predict(dataset_img_test)
print(classification_report(y_test_array, np.argmax(pred,axis=1)))